# DataFrames: Reading in messy data
     
In the [01-data-access](./01-data-access.ipynb) example we show how Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  One key difference, when using Dask Dataframes is that instead of opening a single file with a function like [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), we typically open many files at once with [dask.dataframe.read_csv](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.read_csv). This enables us to treat a collection of files as a single dataset. Most of the time this works really well. But real data is messy and in this notebook we will explore a more advanced technique to bring messy datasets into a dask dataframe.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.1.0.4/10701/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [2]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [3]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name, index=False);

['/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-01.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-02.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-03.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-04.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-05.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-06.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-07.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-08.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-09.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-10.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-11.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-12.csv',
 '/home/runner/work/dask-exa

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [4]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [5]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [6]:
df.head()

,id,name,x,y
0,931,Dan,-0.525556,0.681522
1,1043,Norbert,0.473571,0.708284
2,1000,Edith,0.286195,0.715274
3,1047,Michael,0.534191,-0.918649
4,1010,Dan,-0.208370,0.279540


Let's look at some statistics on the data

In [7]:
df.describe().compute()

,id,x,y
count,2.592000e+06,2.592000e+06,2.592000e+06
mean,9.999660e+02,3.291972e-04,1.740129e-04
std,3.160278e+01,5.775306e-01,5.774119e-01
min,8.420000e+02,-9.999998e-01,-9.999998e-01
25%,9.790000e+02,-4.937873e-01,-4.954257e-01
50%,1.000000e+03,5.996743e-03,7.910481e-03
75%,1.021000e+03,5.069837e-01,5.077860e-01
max,1.164000e+03,9.999985e-01,9.999999e-01


# Make some messy data

Now this works great, and in most cases dd.read_csv or dd.read_parquet etc are the preferred way to read in large collections of data files into a dask dataframe, but real world data is often very messy and some files may be broken or badly formatted. To simulate this we are going to create some fake messy data by tweaking our example csv files. For the file `data/2000-01-05.csv` we will replace with no data and for the file `data/2000-01-07.csv` we will remove the `y` column 

In [8]:
# corrupt the data in data/2000-01-05.csv
with open('data/2000-01-05.csv', 'w') as f:
    f.write('')

In [9]:
# remove y column from data/2000-01-07.csv
import pandas as pd
df = pd.read_csv('data/2000-01-07.csv')
del df['y']
df.to_csv('data/2000-01-07.csv', index=False)

In [10]:
!head data/2000-01-05.csv

In [11]:
!head data/2000-01-07.csv

id,name,x
1024,Hannah,0.8540470135582687
1014,Ingrid,-0.06091756505236012
1010,Patricia,0.17779770093952665
1077,Laura,-0.8783064903812723
974,Wendy,0.2968917343905886
982,Yvonne,-0.913841589378358
983,Quinn,0.4647597031766499
1010,Dan,-0.9116694904615372
1002,Frank,-0.05890834889583485


# Reading the messy data

Let's try reading in the collection of files again

In [12]:
df = dd.read_csv('data/2000-*-*.csv')

In [13]:
df.head()

,id,name,x,y
0,931,Dan,-0.525556,0.681522
1,1043,Norbert,0.473571,0.708284
2,1000,Edith,0.286195,0.715274
3,1047,Michael,0.534191,-0.918649
4,1010,Dan,-0.208370,0.279540


Ok this looks like it worked, let us calculate the dataset statistics again

In [14]:
df.describe().compute()

distributed.worker - WARNING -  Compute Failed
Function:  execute_task
args:      ((subgraph_callable, (<function pandas_read_text at 0x7f53f6303ee0>, <function _make_parser_function.<locals>.parser_f at 0x7f5408458ee0>, (<function read_block_from_file at 0x7f53f40c88b0>, <OpenFile '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-07.csv'>, 0, 64000000, b'\n'), b'id,name,x,y\n', {}, {'id': dtype('int64'), 'name': dtype('O'), 'x': dtype('float64'), 'y': dtype('float64')}, ['id', 'name', 'x', 'y'], False, False, None), 'read-csv-db561331e0984f17fbe03b31f14f6803'))
kwargs:    {}
Exception: ValueError('Length mismatch: Expected axis has 3 elements, new values have 4 elements')



ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements

So what happened? 

When creating a dask dataframe from a collection of files, dd.read_csv samples the first few files in the dataset to determine the datatypes and columns available. Since it has not opened all the files it does not now if some of them are corrupt. Hence, `df.head()` works since it is only looking at the first file. `df.describe.compute()` fails because of the corrupt data in `data/2000-01-05.csv`

# Building a delayed reader

To get around this problem we are going to use a more advanced technique to build our dask dataframe. This method can also be used any time some custom logic is required when reading each file. Essentially, we are going to build a function that uses  pandas and some error checking and returns a pandas dataframe. If we find a bad data file we will either find a way to fix/clean the data or we will return and empty pandas dataframe with the same structure as the good data.

In [15]:
import numpy as np
import io

def read_data(filename):
    
    # for this to work we need to explicitly set the datatypes of our pandas dataframe 
    dtypes = {'id': int, 'name': str, 'x': float, 'y': float}
    try:
        # try reading in the data with pandas 
        df = pd.read_csv(filename, dtype=dtypes)
    except:
        # if this fails create an empty pandas dataframe with the same dtypes as the good data
        df = pd.read_csv(io.StringIO(''), names=dtypes.keys(), dtype=dtypes)
    
    # for the case with the missing column, add a column of data with NaN's
    if 'y' not in df.columns:
        df['y'] = np.NaN
        
    return df

Let's test this function on a good file and the two bad files

In [16]:
# test function on a normal file
read_data('data/2000-01-01.csv').head()

,id,name,x,y
0,931,Dan,-0.525556,0.681522
1,1043,Norbert,0.473571,0.708284
2,1000,Edith,0.286195,0.715274
3,1047,Michael,0.534191,-0.918649
4,1010,Dan,-0.208370,0.279540


In [17]:
# test function on the empty file
read_data('data/2000-01-05.csv').head()

,id,name,x,y


In [18]:
# test function on the file missing the y column
read_data('data/2000-01-07.csv').head()

,id,name,x,y
0,1024,Hannah,0.854047,NaN
1,1014,Ingrid,-0.060918,NaN
2,1010,Patricia,0.177798,NaN
3,1077,Laura,-0.878306,NaN
4,974,Wendy,0.296892,NaN


# Assembling the dask dataframe

First we take our `read_data` function and convert it to a dask delayed function

In [19]:
from dask import delayed
read_data = delayed(read_data)

Let us look at what the function does now

In [20]:
df = read_data('data/2000-01-01.csv')
df

Delayed('read_data-6abc034f-d006-41d3-9f9e-78c5f044cf25')

It creates a delayed object, to actually run read the file we need to run `.compute()`

In [21]:
df.compute()

,id,name,x,y
0,931,Dan,-0.525556,0.681522
1,1043,Norbert,0.473571,0.708284
2,1000,Edith,0.286195,0.715274
3,1047,Michael,0.534191,-0.918649
4,1010,Dan,-0.208370,0.279540
...,...,...,...,...
86395,954,Yvonne,0.186038,0.277587
86396,991,Kevin,-0.686581,-0.994061
86397,999,Norbert,0.687327,-0.930196
86398,1025,Ingrid,0.305286,-0.261965


Now let's build a list of all the available csv files

In [22]:
# loop over all the files
from glob import glob
files = glob('data/2000-*-*.csv')
files

['data/2000-01-13.csv',
 'data/2000-01-28.csv',
 'data/2000-01-01.csv',
 'data/2000-01-08.csv',
 'data/2000-01-20.csv',
 'data/2000-01-12.csv',
 'data/2000-01-25.csv',
 'data/2000-01-30.csv',
 'data/2000-01-16.csv',
 'data/2000-01-11.csv',
 'data/2000-01-17.csv',
 'data/2000-01-27.csv',
 'data/2000-01-09.csv',
 'data/2000-01-18.csv',
 'data/2000-01-19.csv',
 'data/2000-01-26.csv',
 'data/2000-01-23.csv',
 'data/2000-01-05.csv',
 'data/2000-01-06.csv',
 'data/2000-01-03.csv',
 'data/2000-01-04.csv',
 'data/2000-01-02.csv',
 'data/2000-01-14.csv',
 'data/2000-01-24.csv',
 'data/2000-01-29.csv',
 'data/2000-01-21.csv',
 'data/2000-01-10.csv',
 'data/2000-01-15.csv',
 'data/2000-01-07.csv',
 'data/2000-01-22.csv']

Now we run the delayed read_data function on each file in the list

In [23]:
df = [read_data(file) for file in files]
df

[Delayed('read_data-823f5bba-8dca-4f31-9263-c74fe2fa7c26'),
 Delayed('read_data-cf43bb54-2f77-48db-836d-23a9ec517fb2'),
 Delayed('read_data-40d5e9df-6e14-4a1e-a1dd-c7ee0f3522b3'),
 Delayed('read_data-28ecefe6-39d0-4ba5-9f9a-b6369dec5137'),
 Delayed('read_data-ae6b5dd4-00bc-4cc6-bf45-03590cd0f543'),
 Delayed('read_data-c3a0c053-b83f-4076-bcd1-ef237605d86c'),
 Delayed('read_data-7c4e1b41-5ca2-4321-90ef-a58be076da5e'),
 Delayed('read_data-32b84ffd-2190-48af-b2fa-1b69dbdc6851'),
 Delayed('read_data-4cc49e26-71b4-4633-b261-93e38775b7a2'),
 Delayed('read_data-e7e7d9f0-8f6c-4bb9-8951-77e6418f6e35'),
 Delayed('read_data-71ecf9f0-9356-4acf-a898-9946d7f344d7'),
 Delayed('read_data-4543104e-a726-4f2b-8436-198d15c794e1'),
 Delayed('read_data-938bf9de-4c8b-4b03-9442-36130b84a0b3'),
 Delayed('read_data-fdc6a612-ddc2-4548-9eca-6ffee3028ca4'),
 Delayed('read_data-31ddcd70-39da-459f-abfa-d13eb628b99a'),
 Delayed('read_data-90381b77-8563-41df-9d7e-c69ab5e43606'),
 Delayed('read_data-5f394d2b-4026-4e24-b

Then we use [dask.dataframe.from_delayed](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.from_delayed). This function creates a Dask DataFrame from a list of delayed objects as long as each delayed object returns a pandas dataframe. The structure of each individual dataframe returned must also be the same.

In [24]:
df = dd.from_delayed(df, meta={'id': int, 'name': str, 'x': float, 'y': float})
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


Note: we provided the dtypes in the `meta` keyword to explicitly tell Dask Dataframe what kind of dataframe to expect. If we did not do this Dask would infer this from the first delayed object which could be slow if it was a large csv file

## Now let's see if this works

In [25]:
df.head()

,id,name,x,y
0,937,George,-0.009691,0.362562
1,984,Edith,-0.250648,-0.113874
2,1013,Xavier,0.725391,-0.278582
3,959,Ursula,0.371744,-0.822903
4,1019,Norbert,-0.842825,0.013357


In [26]:
df.describe().compute()

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


,id,x,y
count,2.505600e+06,2.505600e+06,2.419200e+06
mean,9.999633e+02,2.459832e-04,2.535097e-04
std,3.160336e+01,5.775142e-01,5.773555e-01
min,8.420000e+02,-9.999998e-01,-9.999995e-01
25%,9.790000e+02,-4.937873e-01,-4.954257e-01
50%,1.000000e+03,5.996743e-03,7.910481e-03
75%,1.021000e+03,5.069837e-01,5.077860e-01
max,1.164000e+03,9.999985e-01,9.999991e-01


## Success!

To recap, in this example, we looked at an approach to create a Dask Dataframe from a collection of many data files. Typically you would use built-in functions like `dd.read_csv` or `dd.read_parquet` to do this. Sometimes, this is not possible because of messy/corrupted files in your dataset or some custom processing that might need to be done. 

In these cases, you can build a Dask DataFrame with the following steps.

1. Create a regular python function that reads the data, performs any transformations, error checking etc and always returns a  Pandas dataframe with the same structure
2. Convert this read function to a delayed object using the `dask.delayed` function
3. Call each file in your dataset with the delayed data reader and assemble the output as a list of delayed objects
4. Used `dd.from_delayed` to covert the list of delayed objects to a Dask Dataframe 

This same technique can be used in other situations as well. Another example might be data files that require using a specialized reader, or several transformations before they can be converted to a pandas dataframe.